In [1]:
!git clone https://github.com/federico2879/MLDL2024_semantic_segmentation.git

Cloning into 'MLDL2024_semantic_segmentation'...
remote: Enumerating objects: 699, done.
remote: Counting objects: 100% (350/350), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 699 (delta 207), reused 295 (delta 174), pack-reused 349
Receiving objects: 100% (699/699), 333.54 KiB | 3.47 MiB/s, done.
Resolving deltas: 100% (414/414), done.


In [2]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms.functional import InterpolationMode
from torch.utils.data import DataLoader
import numpy as np
from MLDL2024_semantic_segmentation.datasets.cityscapes import CityScapes
from MLDL2024_semantic_segmentation.datasets.gta5 import GTA5
from MLDL2024_semantic_segmentation.models.bisenet.build_bisenet import *
from MLDL2024_semantic_segmentation.train import *
from MLDL2024_semantic_segmentation.train_adv import * 
from MLDL2024_semantic_segmentation.utils import *
from MLDL2024_semantic_segmentation.models.IOU import * 
from MLDL2024_semantic_segmentation.models.Adversarial.discriminator import FCDiscriminator

In [3]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
# Setup fixed parameters
num_epochs = 50
num_classes = 19
batch_size = 4

GTA_dim = (1280, 720)
CityScapes_dim = (1024, 512)

In [5]:
# Transformations
transform_image = {
    'GTA': transforms.Compose([
        transforms.Resize(GTA_dim),
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.05),
        transforms.RandomHorizontalFlip(0.15),
        transforms.GaussianBlur(kernel_size=3, sigma=(0.2, 0.8)),
        transforms.ToTensor(),        
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ]),
    'CityScapes': transforms.Compose([
        transforms.Resize(CityScapes_dim),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
}
                   
transform_target = {
    'GTA': transforms.Compose([
        transforms.Resize(GTA_dim, interpolation=InterpolationMode.NEAREST),
    ]),
    'CityScapes': transforms.Compose([
        transforms.Resize(CityScapes_dim, interpolation=InterpolationMode.NEAREST)
    ])
}

In [6]:
# Create dataloaders

dataset_source = GTA5('/kaggle/input/mldl-gta5/GTA', 
                     transform = transform_image['GTA'], 
                     label_transform = transform_target['GTA'])
source_dataloader = DataLoader(dataset_source, batch_size=batch_size, shuffle=True)

dataset_target = CityScapes('/kaggle/input/cityscapes/Cityscapes/Cityspaces', 
                          split = 'train', transform = transform_image['CityScapes'], 
                          label_transform = transform_target['CityScapes'])
target_dataloader = DataLoader(dataset_target, batch_size=batch_size, shuffle=True)

dataset_target_test = CityScapes('/kaggle/input/cityscapes/Cityscapes/Cityspaces', 
                          split = 'val', transform = transform_image['CityScapes'], 
                          label_transform = transform_target['CityScapes'])
target_dataloader_test = DataLoader(dataset_target_test, batch_size=batch_size, shuffle=False)

In [7]:
import torch.nn as nn
import torch.nn.functional as F


import torch.nn as nn
import torch.nn.functional as F


class FCDiscriminator(nn.Module):

    def __init__(self, num_classes, ndf = 64):
        super(FCDiscriminator, self).__init__()

        self.conv1 = nn.Conv2d(num_classes, ndf, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv2d(ndf, ndf*2, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(ndf*2, ndf*4, kernel_size=4, stride=2, padding=1)
        self.conv4 = nn.Conv2d(ndf*4, ndf*8, kernel_size=4, stride=2, padding=1)
        self.classifier = nn.Conv2d(ndf*8, 1, kernel_size=4, stride=2, padding=1)

        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2, inplace=True)
        self.up_sample = nn.Upsample(scale_factor=32, mode='bilinear')
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        x = self.conv1(x)
        x = self.leaky_relu(x)
        x = self.conv2(x)
        x = self.leaky_relu(x)
        x = self.conv3(x)
        x = self.leaky_relu(x)
        x = self.conv4(x)
        x = self.leaky_relu(x)
        x = self.classifier(x)
        x = self.up_sample(x)
        x = self.sigmoid(x) 

        return x

In [8]:
import torch
import torch.nn.functional as F

def train_adv(model, discr, seg_loss, bce_loss, targetloader, sourceloader, optimizer, opt_discr, 
              device, num_classes):
    model.train()
    discr.train()
  
    # Labels for adversarial training
    source_label = 0
    target_label = 1
    
    # Create iterators
    sourceloader_iter = iter(sourceloader)
    targetloader_iter = iter(targetloader)
    
    max_iterations = min(len(targetloader), len(sourceloader))
    
    for idx in range(max_iterations):

        optimizer.zero_grad()
        opt_discr.zero_grad()

        ##### Train G #####

        # Don't accumulate grads in D
        for param in discr.parameters():
            param.requires_grad = False

        # Train with source
        batch = next(sourceloader_iter)
        images, labels = batch
        images = images.to(device)
        labels = labels.squeeze(dim=1).long().to(device)

        pred = model(images)[0]
        
        loss_seg = seg_loss(pred, labels)
        loss_seg.backward()

        # Train with target
        batch = next(targetloader_iter)
        images, _ = batch
        images = images.to(device)

        pred_target = model(images)[0]

        D_out = discr(F.softmax(pred_target, dim=1))

        loss_adv_target = bce_loss(D_out, torch.FloatTensor(D_out.data.size()).fill_(source_label).to(device))
        loss_adv_target.backward()

        ##### Train D #####

        # Bring back requires_grad
        for param in discr.parameters():
            param.requires_grad = True

        # Train with source
        pred = pred.detach()

        D_out = discr(F.softmax(pred, dim=1))

        loss_D = bce_loss(D_out, torch.FloatTensor(D_out.data.size()).fill_(source_label).to(device))
        loss_D.backward()

        # Train with target
        pred_target = pred_target.detach()
        
        D_out = discr(F.softmax(pred_target, dim=1))

        loss_D = bce_loss(D_out, torch.FloatTensor(D_out.data.size()).fill_(target_label).to(device))
        loss_D.backward()

        optimizer.step()
        opt_discr.step()

In [9]:
# Inizialization of the models
generator = BiSeNet(num_classes=num_classes, context_path="resnet18").to(device)
discriminator = FCDiscriminator(num_classes=num_classes).to(device)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 143MB/s] 
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 153MB/s]  


In [10]:
# Optimizers
optimizer_G = torch.optim.SGD(generator.parameters(), lr=2.5e-2, momentum=0.9, weight_decay=1e-4)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=1e-4, betas=(0.9, 0.99))

# Loss functions
segmentation_loss_fn = nn.CrossEntropyLoss(ignore_index=255)
adversarial_loss_fn = nn.BCEWithLogitsLoss()

In [11]:
# metrics
meanIOU = np.zeros((num_epochs,1))
IOU = np.zeros((num_epochs,num_classes))
loss = np.zeros((num_epochs,1))

In [12]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [ ]:
for epoch in range(num_epochs):
    poly_lr_scheduler(optimizer_G, 2.5e-2, epoch, lr_decay_iter=1, max_iter=num_epochs, power=0.9)
    poly_lr_scheduler(optimizer_D, 1e-4, epoch, lr_decay_iter=1, max_iter=num_epochs, power=0.9)
    
    train_adv(generator, discriminator, segmentation_loss_fn, adversarial_loss_fn, target_dataloader, 
              source_dataloader, optimizer_G, optimizer_D, device, num_classes)
    
    meanIOU[epoch], IOU[epoch,:], loss[epoch] = test(generator, target_dataloader_test, segmentation_loss_fn, num_classes)
    print(f"epoch: {epoch + 1}, Validation IOU: {meanIOU[epoch,0]:.2f}")
    
    torch.save({
    'epoch': epoch + 1,
    'state_dict_gen': generator.state_dict(),
    'state_dict_dis': discriminator.state_dict(),
    'optimizer_gen': optimizer_G.state_dict(),
    'optimizer_dis': optimizer_D.state_dict(),
    'meanIOU': meanIOU,
    'IOU': IOU,
    'loss': loss    
    },"checkpoint.pth.tar")

epoch: 1, Validation IOU: 0.17
epoch: 2, Validation IOU: 0.19
epoch: 3, Validation IOU: 0.19
epoch: 4, Validation IOU: 0.21
epoch: 5, Validation IOU: 0.22
epoch: 6, Validation IOU: 0.24
epoch: 7, Validation IOU: 0.21
epoch: 8, Validation IOU: 0.22
epoch: 9, Validation IOU: 0.21
epoch: 10, Validation IOU: 0.20
epoch: 11, Validation IOU: 0.23
epoch: 12, Validation IOU: 0.20
epoch: 13, Validation IOU: 0.22
epoch: 14, Validation IOU: 0.20
epoch: 15, Validation IOU: 0.22
epoch: 16, Validation IOU: 0.23
epoch: 17, Validation IOU: 0.19
epoch: 18, Validation IOU: 0.24
epoch: 19, Validation IOU: 0.21
epoch: 20, Validation IOU: 0.20
epoch: 21, Validation IOU: 0.22
epoch: 22, Validation IOU: 0.21
epoch: 23, Validation IOU: 0.21
epoch: 24, Validation IOU: 0.23
epoch: 25, Validation IOU: 0.20
epoch: 26, Validation IOU: 0.21
epoch: 27, Validation IOU: 0.24
epoch: 28, Validation IOU: 0.19
epoch: 29, Validation IOU: 0.23
epoch: 30, Validation IOU: 0.23
epoch: 31, Validation IOU: 0.21
epoch: 32, Valida

In [13]:
from MLDL2024_semantic_segmentation.load_checkpoint import *

generator, discriminator, optimizer_G, optimizer_D, start_epoch, meanIOU, IOU, loss \
= load_checkpoint_adversarial(generator, discriminator, optimizer_G, optimizer_D, 
                              filename="/kaggle/input/adv-s4-p1/checkpoint.pth.tar")

for epoch in range(start_epoch,num_epochs):
    poly_lr_scheduler(optimizer_G, 2.5e-2, epoch, lr_decay_iter=1,
                      max_iter=num_epochs, power=0.9)
    poly_lr_scheduler(optimizer_D, 1e-4, epoch, lr_decay_iter=1,
                      max_iter=num_epochs, power=0.9)
    train_adv(generator, discriminator, segmentation_loss_fn, adversarial_loss_fn, target_dataloader, 
              source_dataloader, optimizer_G, optimizer_D, device, num_classes)
    
    meanIOU[epoch], IOU[epoch,:], loss[epoch] = test(generator, target_dataloader_test, segmentation_loss_fn, num_classes)
    print(f"epoch: {epoch + 1}, Validation IOU: {meanIOU[epoch,0]:.2f}")
    
    torch.save({
    'epoch': epoch + 1,
    'state_dict_gen': generator.state_dict(),
    'state_dict_dis': discriminator.state_dict(),
    'optimizer_gen': optimizer_G.state_dict(),
    'optimizer_dis': optimizer_D.state_dict(),
    'meanIOU': meanIOU,
    'IOU': IOU,
    'loss': loss    
    },"checkpoint.pth.tar")

epoch: 35, Validation IOU: 0.21
epoch: 36, Validation IOU: 0.21
epoch: 37, Validation IOU: 0.18
epoch: 38, Validation IOU: 0.23
epoch: 39, Validation IOU: 0.20
epoch: 40, Validation IOU: 0.21
epoch: 41, Validation IOU: 0.22
epoch: 42, Validation IOU: 0.23
epoch: 43, Validation IOU: 0.21
epoch: 44, Validation IOU: 0.20
epoch: 45, Validation IOU: 0.23
epoch: 46, Validation IOU: 0.21
epoch: 47, Validation IOU: 0.23
epoch: 48, Validation IOU: 0.23
epoch: 49, Validation IOU: 0.22
epoch: 50, Validation IOU: 0.21


In [14]:
# final print

print(f"Final mIOU: {meanIOU[epoch,0]:.2f}")
print("Final IOU classes")
print(IOU[epoch,:])

Final mIOU: 0.21
Final IOU classes
[0.75756748 0.10417947 0.69510503 0.16644017 0.15921441 0.04638424
 0.00729614 0.01063082 0.58255069 0.16430608 0.46191691 0.07882946
 0.00632881 0.44601786 0.13545487 0.08711545 0.08808932 0.0029727
 0.00077796]


In [15]:
# writing csv
import csv

with open('meanIOU.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(meanIOU)

with open('IOU.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(IOU)

with open('loss.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(loss)